In [3]:
import numpy as np
import tensorflow as tf
import cv2
import os


In [4]:
train_dir = './seg_train/'
test_dir = './seg_test/'
pred_dir = './seg_pred/'

In [5]:
# defining labels
def get_labels(scene_type):
    if scene_type == 'buildings':
        return 0
    elif scene_type == 'forest':
        return 1
    elif scene_type == 'glacier':
        return 2
    elif scene_type == 'mountain':
        return 3
    elif scene_type == 'sea':
        return 4
    elif scene_type == 'street':
        return 5
    else:
        raise KeyError(f'Invalid scene type {scenetype}')

In [6]:
get_labels('forest')

1

# Loading training and test data


In [7]:
X_train = []
y_train = []
for cls in os.listdir(train_dir):
    print(cls)
    for f in os.listdir(train_dir + cls):
        image = cv2.imread(train_dir + cls + '/' + f)
        image_resize = cv2.resize(image, (64, 64))
        X_train.append(image_resize)
        y_train.append(get_labels(cls))
    
X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train.shape, y_train.shape)

buildings
forest
glacier
mountain
sea
street
(14034, 64, 64, 3) (14034,)


In [8]:
# loading test data
X_test = []
y_test = []
for cls in os.listdir(test_dir):
    print(cls)
    for f in os.listdir(test_dir + cls):
        image = cv2.imread(test_dir + cls + '/' + f)
        resize_image = cv2.resize(image, (64, 64))
        X_test.append(resize_image)
        y_test.append(get_labels(cls))
    
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_test.shape, y_test.shape)

buildings
forest
glacier
mountain
sea
street
(3000, 64, 64, 3) (3000,)


# Scalling train and test data

In [9]:
X_test_scaled = []
X_test_scaled = X_test/255.0

In [12]:
X_train_scaled = []
X_train_scaled = X_train/255.0

In [13]:
y_test[1999]

4

In [14]:
y_train[:5]

array([0, 0, 0, 0, 0])

# Training model

In [15]:
# now we will train a Convulation neural network model
# we will use Sequential model
num_classes = 6


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [17]:
model = Sequential([
    layers.experimental.preprocessing.Rescaling(scale=1./255),
    layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

In [18]:
# compile model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [19]:
history = model.fit(X_train_scaled, y_train[:5000], epochs=30, batch_size=256, validation_data=(X_test_scaled, y_test))

ValueError: Data cardinality is ambiguous:
  x sizes: 14034
  y sizes: 5000
Make sure all arrays contain the same number of samples.

In [16]:
type(y_test)

numpy.ndarray

In [ ]:
import pandas as pd
df = pd.DataFrame(history.history)
df.head()

# Plot graphs

In [1]:
import matplotlib.pyplot as plt


In [ ]:
# plot accuracy vs validation accuracy
plt.plot(range(len(df)), df.accuracy, label='Accuracy')
plt.plot(range(len(df)), df.val_accuracy, label='Validation Accuracy')
plt.legend()

In [ ]:
# plot loss vs validation loss
plt.plot(range(len(df)), df.loss, label='Loss')
plt.plot(range(len(df)), df.val_loss, label='Validation Loss')
plt.legend()

# Loading evaluation data

In [ ]:
X_eval = []
test_files = []
for f in os.listdir(eval_dir):
    test_files.append(f)
    image=cv2.imread(eval_dir + f)
    X_eval.append(image)

X_eval = np.array(X_eval)

X_eval_scaled = X_eval

# Making Predictions

In [ ]:
predictions = model.predict(X_eval_scaled)

predictions[:2]

In [ ]:
predictions = [np.argmax(prediction) for prediction in predictions]

In [ ]:
predictions[:4]

In [ ]:
def get_scene_type(label):
    if label == 0:
        return 'buildings'
    elif label == 1:
        return 'forest'
    elif label == 2:
        return 'glacier'
    elif label == 3:
        return 'mountain'
    elif label == 4:
        return 'sea'
    elif label == 2:
        return 'street'


In [ ]:
predictions_with_type = [get_scene_type(element) for element in predictions]

In [ ]:
df = pd.DataFrame()
df['testfiles'] = test_files
df['predictions'] = predictions_with_type
df.head()


In [ ]:
df.to_csv('results.csv', index=False)